In [1]:
import numpy as np
import pandas as pd
import os
import torch
import time, sys
from transformers import AlbertForQuestionAnswering,AlbertTokenizerFast,AutoTokenizer,AutoModelForQuestionAnswering,AutoModel,XLMRobertaForQuestionAnswering,XLMRobertaTokenizerFast,BertTokenizerFast,BertForQuestionAnswering


In [2]:
tokenizer_address = '../input/huggingface-question-answering-models/multilingual/bert-multi-uncased-finetuned-xquadv1'
model_address = '../input/huggingface-question-answering-models/multilingual/bert-multi-uncased-finetuned-xquadv1'

tokenizer = BertTokenizerFast.from_pretrained(tokenizer_address)
# tokenizer.model_max_length = 512 # only for ALBERT

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
train_data = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/train.csv")
dev_data = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/test.csv")
# size = 100 # size if train set used

def preprocess_data(data):
    encodings = tokenizer(list(data["context"]), list(data["question"]), truncation=True, padding=True)

    start_positions = []
    end_positions = []
    id_ = []
    for i in range(len(data["answer_start"])):
        start_positions.append(encodings.char_to_token(i,data["answer_start"][i]))
        end_positions.append(encodings.char_to_token( i,(data["answer_start"][i] + len(data['answer_text'][i]) - 1) ))
        
            
        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
        
        
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    
    return encodings

len(train_data)

In [5]:
train_encodings = preprocess_data(train_data)
dev_encodings = tokenizer(list(dev_data["context"]), list(dev_data["question"]),max_length=512, truncation=True, padding=True)

In [6]:
from torch.utils.data import DataLoader
from transformers import AdamW

class chaiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)
    
train_dataset = chaiDataset(train_encodings)
dev_dataset = chaiDataset(dev_encodings)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=1, shuffle=False)

In [7]:
model = BertForQuestionAnswering.from_pretrained(model_address)

for param in model.bert.parameters():
    param.requires_grad = False

model.to(device)

In [8]:
optim = AdamW(model.parameters(), lr=1e-4)

model.train()

for epoch in range(30):
    epoch_loss=0
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        batch_loss=loss.item()
        optim.step()
        epoch_loss+=batch_loss
    normalized_epoch_loss = epoch_loss/(len(train_loader))
    print("Epoch {} ; Epoch loss: {} ".format(epoch+1,normalized_epoch_loss))

In [ ]:
torch.save(model,"model_2")
# model = torch.load("../input/chaii-torch/model_2")

In [ ]:
model.eval()
output_words,output_id = [],[]
for batch in dev_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)
    start = torch.argmax(outputs["start_logits"])
    end = torch.argmax(outputs["end_logits"])
    output_tokens = tokenizer.convert_ids_to_tokens(input_ids[0][start:end+1])
    output_words.append(tokenizer.convert_tokens_to_string(output_tokens))

In [ ]:
dev_results = pd.DataFrame({"id":dev_data["id"],"PredictionString":output_words})
dev_results.to_csv("submission.csv",index=False)